<a href="https://colab.research.google.com/github/dcatrib/tesePython/blob/main/tese_PyMuPDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install PyMuPDF requests selenium chromedriver-autoinstaller pandas nltk gensim numpy

!apt-get update
!apt-get install -y wget
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!apt-get install -y ./google-chrome-stable_current_amd64.deb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,031 kB]


In [18]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException
import chromedriver_autoinstaller
import requests
import fitz
import os
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from google.colab import files
import numpy as np
from gensim.models import KeyedVectors, Word2Vec
import gensim
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.ar_model import AutoReg


In [19]:
# Configurando o WebDriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Instala automaticamente a versão correta do ChromeDriver
chromedriver_autoinstaller.install()



'/usr/local/lib/python3.10/dist-packages/chromedriver_autoinstaller/129/chromedriver'

In [20]:
# URL para scrape
url = "https://www.bcb.gov.br/en/publications/copomminutes/cronologicos"

# Criar uma nova instância do Chrome driver
driver = webdriver.Chrome(options=chrome_options)
driver.get(url)

# Esperar o seletor específico estar presente
WebDriverWait(driver, 15).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, 'body > app-root > app-root > main > dynamic-comp > div > div > bcb-publicacao > div > div > bcb-ultimaspublicacoes > div'))
)

# Definir o seletor
selector = 'body > app-root > app-root > main > dynamic-comp > div > div > bcb-publicacao > div > div > bcb-ultimaspublicacoes > div'

# Lista para armazenar hrefs, textos e links de download
hrefs_and_texts = []


In [21]:
# Função para recuperar links e botões de download
def retrieve_links():
    try:
        elements = driver.find_elements(By.CSS_SELECTOR, selector)
        link_count = 0  # Inicializa um contador para os links
        for element in elements:
            links = element.find_elements(By.TAG_NAME, 'a')
            for link in links:
                if link_count >= 200:  # Limita a 50 links
                    return  # Sai da função se 50 links foram processados
                href = link.get_attribute('href')
                text = link.text
                if href and text and text[0].isdigit():
                    driver.get(href)
                    # Esperar o botão de download
                    try:
                        download_button = WebDriverWait(driver, 45).until(
                            EC.presence_of_element_located((By.CSS_SELECTOR, '#publicacao > div.col-lg-9.d-flex.flex-column > div > div > div > div.d-flex.flex-column-reverse.flex-sm-row.justify-content-sm-between.align-items-sm-center > div.d-flex.align-items-center.mb-3.mb-sm-0.d-print-none > download > div > div > a'))
                        )
                        download_href = download_button.get_attribute('href')

                        # Fazer download do PDF
                        pdf_response = requests.get(download_href)
                        pdf_filename = f"{text}.pdf"
                        with open(pdf_filename, 'wb') as pdf_file:
                            pdf_file.write(pdf_response.content)

                        # Extrair texto do PDF usando PyMuPDF
                        pdf_text = ''
                        with fitz.open(pdf_filename) as pdf_file:
                            for page in pdf_file:
                                page_text = page.get_text()
                                pdf_text += page_text + '\n'

                        # Limpeza do texto
                        pdf_text = pdf_text.replace('\n', ' ')  # Remove quebras de linha
                        pdf_text = re.sub(r'\s+', ' ', pdf_text)  # Substitui múltiplos espaços por um único
                        pdf_text = pdf_text.strip()  # Remove espaços no início e no fim

                        # Print da primeira linha extraída
                        first_line = pdf_text.split('.')[0]  # Considera a primeira frase
                        #print(f'Primeira linha extraída: {first_line.strip()}')
                        print(link_count)
                    except (NoSuchElementException, TimeoutException):
                        download_href = None
                        pdf_text = None

                    hrefs_and_texts.append((href, text, download_href, pdf_text))
                    link_count += 1  # Incrementa o contador
                    driver.back()
                    WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR, selector)))
    except StaleElementReferenceException:
        retrieve_links()

# Iniciar a recuperação de links
retrieve_links()

# Fechar o driver
driver.quit()


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199


Minhas stopwords e expressoes

In [22]:
# Minhas stopwords
my_stopwords = {
    'January', 'February', 'March', 'April', 'May', 'June', 'July',
    'August', 'September', 'October', 'November', 'December','january', 'february', 'march', 'april', 'may', 'june', 'July',
    'august', 'september', 'october', 'november', 'december',
    'minutes', 'bcb', 'th', 'copom', 'bcbgovbr', 'brasilia', 'pm','roberto'
}

# Array de expressões a serem removidas
expressions_to_remove = [
    'bcb.gov.br Minutes of the Meeting of the Monetary Policy Committee — Copom',
]

fazer o preprocessamento e criar o df

In [23]:
# Baixar as stopwords se ainda não tiver feito
nltk.download('stopwords')

# Criar um DataFrame com os links e textos extraídos
df = pd.DataFrame(hrefs_and_texts, columns=['Link', 'Texto', 'Link de Download', 'Texto do PDF'])

# Carregar stopwords em inglês
stop_words = set(stopwords.words('english'))
stop_words.update(my_stopwords)

# Função para preprocessar o texto
def preprocess_text(text):
    if text:
        # Remover números, pontuação e caracteres especiais
        text = re.sub(r'[0-9]+', '', text)  # Remove números
        # Remover pontuação, exceto parênteses
        text = re.sub(r'[^\w\s()]+', '', text)  # Mantém letras, números, espaços e parênteses
        text = re.sub(r'\s+', ' ', text)  # Substituir múltiplos espaços por um único espaço
        text = text.strip().lower()  # Retorna texto em minúsculas

        # Remover expressões específicas
        for expr in expressions_to_remove:
            text = text.replace(expr, '')

        # Remover stopwords
        text = ' '.join([word for word in text.split() if word not in stop_words])
        return text
    return ''

# Criar a nova coluna 'Texto Processado'
df['Texto Processado'] = df['Texto do PDF'].apply(preprocess_text)

# Imprimir o DataFrame
print(df)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                                                  Link  \
0    https://www.bcb.gov.br/en/publications/copommi...   
1    https://www.bcb.gov.br/en/publications/copommi...   
2    https://www.bcb.gov.br/en/publications/copommi...   
3    https://www.bcb.gov.br/en/publications/copommi...   
4    https://www.bcb.gov.br/en/publications/copommi...   
..                                                 ...   
195  https://www.bcb.gov.br/en/publications/copommi...   
196  https://www.bcb.gov.br/en/publications/copommi...   
197  https://www.bcb.gov.br/en/publications/copommi...   
198  https://www.bcb.gov.br/en/publications/copommi...   
199  https://www.bcb.gov.br/en/publications/copommi...   

                                     Texto  \
0    265th Meeting - September 17-18, 2024   
1         264th Meeting - July 30-31, 2024   
2         263rd Meeting - June 18-19, 2024   
3            262nd Meeting - May 7-8, 2024   
4        261st Meeting - March 19-20, 2024   
..                         

In [24]:
# Função para extrair ano e mês
def extract_year_month(text):
    # Caso 1: Formato com dia e mês
    match_full = re.search(r'(\w+)\s+\d{1,2}[- ]\d{1,2},\s*(\d{4})', text)
    # Caso 2: Formato apenas com mês e ano
    match_month_year = re.search(r'(\w+)\s+(\d{4})', text)
    # Caso 3: Formato com intervalo de datas
    match_interval = re.search(r'(\w+)\s+\d{1,2}\s*-\s*(\w+)\s+\d{1,2},\s*(\d{4})', text)

    if match_full:
        month_str = match_full.group(1)
        year = match_full.group(2)
    elif match_month_year:
        month_str = match_month_year.group(1)
        year = match_month_year.group(2)
    elif match_interval:
        month_str = match_interval.group(2)  # Captura o mês da última data
        year = match_interval.group(3)
    else:
        return None

    # Converter mês para número
    month_num = pd.to_datetime(month_str, format='%B').month
    return f"{year}-{month_num:02d}"  # Formatar como YYYY-MM

# Criar nova coluna com ano e mês
df['Ano_Mes'] = df['Texto'].apply(extract_year_month)

# Exibir o DataFrame resultante
#print(df)

In [25]:
# Definir as expressões que você deseja encontrar
expressions = [
    r'(?i)(A\)\s*update (?:e conomic|economic) outlook (?:c opoms|copoms) (?:reference|baseline|)\s*scenario)(.*)',
    r'(?i)(Recent Economic Developments)(.*)',
    r'(?i)(Recent Evolution Inflation)(.*)',
]

# Função para extrair texto a partir de várias expressões
def extract_from_expressions(text):
    for pattern in expressions:
        match = re.search(pattern, text)
        if match:
            # Retorna o texto a partir da expressão encontrada até o final
            return text[match.start():]  # A partir do início da expressão até o final do texto
    return text  # Retorna todo o texto se nenhuma expressão for encontrada

df['Texto Processado'] = df['Texto Processado'].apply(extract_from_expressions)


# Definir os títulos das seções, permitindo variações com ou sem espaço
section_titles = [
    r'(?i)A\)\s*update (?:e conomic|economic) outlook (?:c opoms|copoms) (?:reference|baseline|)\s*scenario.*?(?=B\)\s*risks around|B\)\s*scenarios risk analysis|$)',
    r'(?i)(B\)\s*risks around|B\)\s*scenarios risk analysis).*?(?=C\)\s*discussion conduct monetary policy|$)',
    r'(?i)C\)\s*discussion conduct monetary policy.*?(?=D\)\s*monetary policy decision|$)',
    r'(?i)D\)\s*monetary policy decision.*?(?=Footnotes|$)',
    r'(?i)Footnotes.*'  # Captura tudo até o final do texto
]


# Criar variáveis para cada seção no DataFrame
for title in section_titles:
    # Usar str.extract para capturar as seções
    section_name = re.search(r'A|B|C|D|Footnotes', title).group(0)  # Captura A, B, C, D ou Footnotes
    df[section_name] = df['Texto Processado'].str.extract(f'({title})')[0]

print (df[['Texto Processado']])
# Exibir o DataFrame resultante
#print(df[['A', 'B', 'C', 'D', 'Footnotes']])

                                      Texto Processado
0    a) update economic outlook copoms scenario glo...
1    a) update economic outlook copoms scenario glo...
2    a) update economic outlook copoms scenario glo...
3    a) update economic outlook copoms scenario glo...
4    a) update economic outlook copoms scenario glo...
..                                                 ...
195  meeting banco central brasil monetary policy c...
196  information unrestricted disclosure intended b...
197  meeting banco central brasil monetary policy c...
198  meeting banco central brasil monetary policy c...
199  rd meeting banco central brasil monetary polic...

[200 rows x 1 columns]


Carregar a base dos pibs trimestrais

In [26]:
# https://www.ibge.gov.br/estatisticas/economicas/contas-nacionais/9300-contas-nacionais-trimestrais.html?=&t=series-historicas&utm_source=landing&utm_medium=explica&utm_campaign=pib#evolucao-taxa

# Fazendo upload do arquivo CSV
uploaded = files.upload()

# Carregar o arquivo CSV usando o delimitador correto
df_trimestres = pd.read_csv('pib_trimestral.csv', delimiter=';', header=1)  # O cabeçalho correto

# Excluir a primeira coluna se necessário (baseado no que você descreveu)
df_trimestres = df_trimestres.iloc[:1, 1:]  # Remove a primeira coluna


# Verifique o DataFrame após as alterações
print("DataFrame após a leitura:")
print(df_trimestres.head())

# Agora, precisamos reorganizar os dados
# Vamos usar melt para transformar os dados
df_trimestres_melted = df_trimestres.melt(var_name='Trimestre', value_name='Valor')

# Verifique o DataFrame derretido
print("DataFrame derretido:")
print(df_trimestres_melted.head())

# Função para gerar mapeamento de trimestres
def gerar_mapeamento_trimestres(inicio_ano, fim_ano):
    mapeamento = {}
    for ano in range(inicio_ano, fim_ano + 1):
        for trimestre in range(1, 5):
            mes = trimestre * 3  # 1: Março, 2: Junho, 3: Setembro, 4: Dezembro
            ano_mes = f"{ano}-{mes:02d}"
            mapeamento[f"{trimestre}º trimestre {ano}"] = ano_mes
    return mapeamento

# Gerar mapeamento de 1996 a 2024
mapeamento_trimestre = gerar_mapeamento_trimestres(1996, 2024)
df_trimestres_melted['Ano_Mes'] = df_trimestres_melted['Trimestre'].map(mapeamento_trimestre)

print("DataFrame com o mapeamento:")
print(df_trimestres_melted[['Trimestre', 'Ano_Mes']].head(20))  # Exibir as primeiras 20 linhas

# Verificar se há valores ausentes na coluna Ano_Mes
valores_ausentes = df_trimestres_melted['Ano_Mes'].isna().sum()
print(f"Número de valores ausentes em 'Ano_Mes': {valores_ausentes}")

# Exibir trimestres e seus respectivos Ano_Mes
for trimestre in df_trimestres_melted['Trimestre'].unique():
    print(f"{trimestre}: {df_trimestres_melted[df_trimestres_melted['Trimestre'] == trimestre]['Ano_Mes'].values}")


# Verifique o resultado do mapeamento
print("Trimestres únicos:")
print(df_trimestres_melted['Trimestre'].unique())
print(df_trimestres_melted[['Ano_Mes', 'Valor']])

print(df.head())

# Agora, mescle os dois dataframes (supondo que o df original esteja carregado em df)
df_final = pd.merge(df, df_trimestres_melted[['Ano_Mes', 'Valor']], on='Ano_Mes', how='outer')
df_final = df_final.dropna(subset=['Texto Processado'])

# Verifique o resultado final
print(df_final.head())

Saving pib_trimestral.csv to pib_trimestral (1).csv
DataFrame após a leitura:
   1º trimestre 1996  2º trimestre 1996  3º trimestre 1996  4º trimestre 1996  \
0                2.5                2.1                2.3                2.2   

   1º trimestre 1997  2º trimestre 1997  3º trimestre 1997  4º trimestre 1997  \
0                3.2                4.0                3.0                3.4   

   1º trimestre 1998  2º trimestre 1998  ...  1º trimestre 2022  \
0                2.8                2.0  ...                4.7   

   2º trimestre 2022  3º trimestre 2022  4º trimestre 2022  1º trimestre 2023  \
0                2.7                2.7                3.0                3.7   

   2º trimestre 2023  3º trimestre 2023  4º trimestre 2023  1º trimestre 2024  \
0                3.7                3.1                2.9                2.5   

   2º trimestre 2024  
0                2.5  

[1 rows x 114 columns]
DataFrame derretido:
           Trimestre  Valor
0  1º trimestre 

Treinando o modelo Word2vec. Utilizando o Texto Processado como corpus

In [11]:
# Carregar o CSV (Para nao precisar rodar tudo novamente)
#df = pd.read_csv('seu_arquivo.csv')  # Substitua pelo caminho do seu arquivo
# Extraindo a quarta coluna como corpus
#corpus = df.iloc[:, 3].tolist()  # Altere o índice se necessário

# Extraindo a quarta coluna como corpus
corpus = df['Texto Processado'].tolist()  # Altere o índice se necessário

# Pré-processamento do texto (divisão em palavras)
# Aqui estamos usando um simples split, mas você pode querer fazer mais pré-processamento
corpus_tokenized = [text.split() for text in corpus]

# Word2Vec (cbow)
print("Treinando Word2Vec...")
word2vec_model = gensim.models.Word2Vec(corpus_tokenized, vector_size=100, window=5, min_count=1, workers=4)

Treinando Word2Vec...


Criando o vetor medio dos vetores medios das atas referente ao trimestre

In [28]:
# Função para obter o vetor médio
def get_average_vector(text):
    words = text.split()
    word_vectors = [word2vec_model.wv[word] for word in words if word in word2vec_model.wv]
    if word_vectors:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(word2vec_model.vector_size)  # Retorna vetor zero se não houver palavras válidas

# Criar a nova coluna 'Texto Processado'
df_final['Vetor Médio'] = df_final['Texto Processado'].apply(get_average_vector)

# Inicializa uma lista para os vetores médios do trimestre
vetores_medio_trimeste = []
previous_vectors = []  # Lista para acumular vetores das linhas com valor missing

# Para cada linha no DataFrame
for i in range(len(df_final)):
    # Se o valor não for missing
    if not np.isnan(df_final['Valor'].iloc[i]):
        # Se houver vetores acumulados, calcula a média
        if previous_vectors:
            vetor_medio_acumulado = np.mean(previous_vectors, axis=0)
            # Calcula a média entre o vetor acumulado e o vetor atual
            vetor_medio_final = np.mean([vetor_medio_acumulado, df_final['Vetor Médio'].iloc[i]], axis=0)
            vetores_medio_trimeste.append(vetor_medio_final)
        else:
            # Caso não haja vetores acumulados, apenas adiciona o vetor atual
            vetores_medio_trimeste.append(df_final['Vetor Médio'].iloc[i])

        # Limpa a lista acumulada para o próximo conjunto
        previous_vectors = []
    else:
        # Se o valor for missing, acumula o vetor médio
        previous_vectors.append(df_final['Vetor Médio'].iloc[i])
        vetores_medio_trimeste.append(None)  # Adiciona None para os missing

# Adiciona a nova coluna ao DataFrame
df_final['vetor_medio_trimestre'] = vetores_medio_trimeste

# Verifica o resultado
print(df_final[['Texto Processado', 'Valor', 'Vetor Médio', 'vetor_medio_trimestre']].head(20))

                                     Texto Processado  Valor  \
24  rd meeting banco central brasil monetary polic...    NaN   
25  meeting banco central brasil monetary policy c...    NaN   
26  meeting banco central brasil monetary policy c...    0.7   
27  meeting banco central brasil monetary policy c...    NaN   
28  information unrestricted disclosure intended b...    NaN   
29  nd meeting banco central brasil (bcb) monetary...    0.7   
30  information unrestricted disclosure intended b...    0.7   
31  july rd meeting banco central brasil (bcb) mon...    NaN   
32  banco central brasil (bcb) meeting monetary po...    NaN   
33  banco central brasil (bcb) meeting monetary po...    1.6   
34  st banco central brasil (bcb) meeting monetary...    NaN   
35  banco central brasil (bcb) meeting monetary po...    NaN   
36  banco central brasil (bcb) meeting monetary po...    NaN   
37  banco central brasil (bcb) meeting monetary po...    3.1   
38  banco central brasil meeting monetar

Previsao

In [29]:
# Descartar as observações onde a coluna 'Valor' é missing (NaN)
df_final_cleaned = df_final.dropna(subset=['Valor'])

# Verificar o resultado
#print(df_final_cleaned.head())  # Exibe as primeiras linhas do DataFrame limpo

# Calcular o vetor médio para cada texto processado
X = np.array([get_average_vector(text) for text in df_final_cleaned['Texto Processado']])

# Y: Valores a serem previstos
y = df_final_cleaned['Valor'].values

# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Treinando o modelo de regressão
model = LinearRegression()
model.fit(X_train, y_train)

# Fazer previsões
predictions = model.predict(X_test)

# Verificar as previsões
print(predictions)

# Criar um DataFrame para comparar previsões e valores reais
results_df = pd.DataFrame({
    'Valor Real': y_test,
    'Previsão': predictions
})

# Verificar os resultados
print(results_df)

# Calcular o erro médio quadrático (MSE)
mse = mean_squared_error(y_test, predictions)

# Imprimir o MSE
print(f'Mean Squared Error: {mse}')

# Ajustar um modelo autoregressivo (AR) usando apenas y
model_ar2 = AutoReg(y_train, lags=1)  # Use o número de lags apropriado
model_fit_ar2 = model_ar2.fit()

# Fazer previsões
predictions_ar2 = model_fit_ar2.predict(start=len(y_train), end=len(y_train) + len(y_test) - 1)

# Calcular o erro médio quadrático (MSE)
mse_ar2 = mean_squared_error(y_test, predictions_ar2)

# Imprimir o MSE
print(f'Mean Squared Error AR2: {mse_ar2}')

[ 3.2633877   3.3016148   6.26202    -0.2717247   1.7239513   5.4998417
  4.5408688   5.838331    4.886341    5.2608852   5.6140385   4.382736
  2.7309132  -1.8716412   0.84503174  5.2882595  -3.4165096   1.7729893
  4.46159     2.5309525   1.7208157   5.0216293   4.8089848 ]
    Valor Real  Previsão
0          3.1  3.263388
1          4.8  3.301615
2          4.2  6.262020
3          0.7 -0.271725
4         -1.2  1.723951
5          2.5  5.499842
6          2.8  4.540869
7          6.6  5.838331
8          5.8  4.886341
9          1.2  5.260885
10         2.7  5.614038
11         5.3  4.382736
12         1.4  2.730913
13        -4.5 -1.871641
14         3.6  0.845032
15         6.3  5.288260
16        -1.7 -3.416510
17         2.0  1.772989
18         7.5  4.461590
19         1.2  2.530952
20         3.1  1.720816
21         3.2  5.021629
22         5.6  4.808985
Mean Squared Error: 4.018541577572664
Mean Squared Error AR2: 8.130902685223948


In [14]:
#exportando para um arquivo csv

output_csv = 'dados_extraidos.csv'  # Nome do arquivo CSV
df_final.to_csv(output_csv, index=False)

# Se você estiver usando Google Colab, faça o download do arquivo CSV
try:
    from google.colab import files
    files.download(output_csv)
except ImportError:
    print("Não é um ambiente Colab. O arquivo está salvo como:", output_csv)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

--------------------------------------------------------------------------------

Sintonia fina de base pre treinada

In [27]:
import numpy as np
from gensim.models import KeyedVectors, Word2Vec
import os
from google.colab import drive

drive.mount('/content/drive')

# Defina o caminho para os embeddings
embeddings_path = '/content/drive/My Drive/Caen/TESE/word2vec-google-news-300.model.vectors.npy'

# Carregar os vetores
vectors = np.load(embeddings_path)

# Tokenizar cada frase do seu DataFrame
corpus = df['Texto Processado'].tolist()
tokenized_corpus = [sentence.split() for sentence in corpus]  # Tokenização simples

# Criar um novo modelo Word2Vec
word2vec_model = Word2Vec(vector_size=300, window=5, min_count=1, sg=1)

# Construir o vocabulário do corpus
word2vec_model.build_vocab(tokenized_corpus)

# Inicializar os vetores do novo modelo com os vetores pré-treinados
for word in word2vec_model.wv.index_to_key:
    if word in vectors:  # Verifique se a palavra existe nos vetores
        word2vec_model.wv[word] = vectors[vectors.index(word)]  # Atribua o vetor pré-treinado

# Treinar o modelo no seu conjunto de dados
word2vec_model.train(tokenized_corpus, total_examples=len(tokenized_corpus), epochs=new_model.epochs)

# Salvar o novo modelo
word2vec_model.save("/content/drive/My Drive/Caen/TESE/meu_modelo_fine_tuned_word2vec.model")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


(2376512, 2650340)

sintonia fina do w2vec a partir de uma base pre treinada(ainda nao funciona!!!)

---



In [1]:
# sintonia fina partindo do modelo pre treinado
import numpy as np
from gensim.models import Word2Vec

# Defina o caminho para o modelo pré-treinado
model_path = '/content/drive/My Drive/Caen/TESE/word2vec-google-news-300.model'

# Carregar o modelo Word2Vec pré-treinado
pretrained_model = Word2Vec.load(model_path)

# Tokenizar cada frase do seu DataFrame
corpus = df['Texto Processado'].tolist()
tokenized_corpus = [sentence.split() for sentence in corpus]  # Tokenização simples

# Atualizar o vocabulário do modelo pré-treinado com seu corpus
pretrained_model.build_vocab(tokenized_corpus, update=True)

# Treinar o modelo com seu corpus
pretrained_model.train(tokenized_corpus, total_examples=len(tokenized_corpus), epochs=pretrained_model.epochs)

# Salvar o novo modelo ajustado
#pretrained_model.save("/content/drive/My Drive/Caen/TESE/meu_modelo_fine_tuned_word2vec.model")

AttributeError: Model of type <class 'gensim.models.keyedvectors.KeyedVectors'> can't be loaded by <class 'gensim.models.word2vec.Word2Vec'>

In [35]:
import numpy as np
import os
from google.colab import drive
drive.mount('/content/drive')


# Listar arquivos no diretório onde o arquivo deveria estar
print(os.listdir('/content/drive/My Drive/Caen/TESE'))

# Defina o caminho para o arquivo no Google Drive
embeddings_path = '/content/drive/My Drive/Caen/TESE/word2vec-google-news-300.model.vectors.npy'
embeddings = np.load(embeddings_path)

# Carregar os tokens
tokens_path = '/content/drive/My Drive/Caen/TESE/word2vec-google-news-300.model.vectors.txt'
with open(tokens_path) as fp:
    tokens = [line.strip() for line in fp]

# Acessar o embedding da palavra 'hello'
if 'hello' in tokens:
    hello_embedding = embeddings[tokens.index('hello')]
    print(hello_embedding)
else:
    print("A palavra 'hello' não está nos tokens.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['tese PyMuPDF.ipynb', 'Antigos', 'word2vec-google-news-300.model.vectors.npy', 'Untitled0.ipynb', 'Proximos passos.gdoc']


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/Caen/TESE/word2vec-google-news-300.model.vectors.txt'

In [ ]:
# Faça o upload do arquivo .npy
uploaded = files.upload()

# Carregar o arquivo .npy
vectors_path = 'word2vec-google-news-300.model.vectors.npy'  # Altere se necessário
vectors = np.load(vectors_path)

# Carregar as palavras associadas
words_path = 'path/to/word2vec-google-news-300.model.vectors.words'  # Altere conforme necessário
with open(words_path, 'r', encoding='utf-8') as f:
    words = f.read().splitlines()

# Extraindo a coluna como corpus
corpus = df['Texto Processado'].tolist()

# Supondo que o corpus seja uma lista de strings, tokenize cada frase
tokenized_corpus = [sentence.split() for sentence in corpus]  # Tokenização simples

#Criar um modelo KeyedVectors
word2vec_model = KeyedVectors(vector_size=300)
word2vec_model.add_vectors(words, vectors)

# Criar um novo modelo Word2Vec e transferir pesos
new_model = Word2Vec(tokenized_corpus, vector_size=300, window=5, min_count=1, sg=1)

# Inicializar os pesos do modelo com os vetores pré-treinados
for word in word2vec_model.key_to_index:
    if word in new_model.wv.key_to_index:
        new_model.wv[word] = word2vec_model[word]

# Treinar o modelo no seu conjunto de dados
new_model.train(tokenized_corpus, total_examples=len(tokenized_corpus), epochs=new_model.epochs)

# Salvar o novo modelo
new_model.save("meu_modelo_fine_tuned_word2vec.model")

files.download("meu_modelo_fine_tuned_word2vec.model")

KeyboardInterrupt: 